## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_2_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_exec_csv/"+'gcc6'+"_2_32_exec.csv", index_col=0)

# 형태 출력
print(gcc6_2_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_2_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_2_32.head()

(11076816, 2)
reset_index 완료
input data shape


,bin,label
0,53,1
1,83,0
2,ec,0
3,08,0
4,e8,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_2_32['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_2_32['label'].value_counts())

256
0    11040011
1       36805
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
######################## 
idx = gcc6_2_32[gcc6_2_32['label']==1].index  # 407, 474 ...
ls = list(idx)

# 최종 뽑을 행에 대한 index
ls_idx = [] 
left_idx, right_idx = 0, 48 # 3개씩

# 6gram
for k in range(left_idx, right_idx):
    ls_idx.extend(list(idx + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx = list(filter(lambda x: x<len(gcc6_2_32), ls_idx))
print(len(ls_idx))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub = len(ls_idx)%(right_idx)
print('나머지', sub)

ls_idx = ls_idx[:len(ls_idx)-sub]
print('최종 길이', len(ls_idx))

print('gcc6_2_32', len(ls_idx))

# loc 로 수정필요
gcc6_2_32_Ngram = gcc6_2_32.loc[ls_idx,:].copy()

1766604
나머지 12
최종 길이 1766592
gcc6_2_32 1766592


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성
# random index
random_idx = np.random.randint(len(gcc6_2_32)-right_idx)

# 목표치
goal = len(gcc6_2_32_Ngram)/right_idx
count=0

d = pd.DataFrame(columns=gcc6_2_32.columns)

# goal 에 도달할 때까지
while count!=goal:
    
    # 진행상황 살펴보기 위함
    if count%1000==0:
        print(count, end=' ')
        
    # 랜덤 N 바이트씩 뽑음
    df = gcc6_2_32[random_idx:random_idx+right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df['label']:
        d = pd.concat([d, df])
        count+=1

print('완료')

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 완료


## (5) False Data + True Data 합치기

In [5]:
ff = pd.DataFrame(columns=gcc6_2_32.columns)

for i in range(0, int(len(d)/right_idx)):
    ff = pd.concat([ff, gcc6_2_32[i*right_idx:(i+1)*right_idx], d[i*right_idx:(i+1)*right_idx]])
    
    if i%1000==0:
        print(i, end=' ')
ff.shape

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 

(3533184, 2)

## (6) one hot encoding

In [6]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_2_32_onehot_Ngram = pd.get_dummies(ff['bin'])
gcc6_2_32_onehot_Ngram = pd.concat([ff['label'], gcc6_2_32_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(gcc6_2_32_onehot_Ngram.shape)

원핫인코딩완료
(3533184, 257)


In [7]:
# 훈련 데이터, 훈련 라벨
x_gcc6_2_32_3 = gcc6_2_32_onehot_Ngram.iloc[:,1:].to_numpy()
y_gcc6_2_32_3 = gcc6_2_32_onehot_Ngram['label'].to_numpy()
print(x_gcc6_2_32_3.shape, y_gcc6_2_32_3.shape)

x_gcc6_2_32_3 = x_gcc6_2_32_3.reshape(-1, right_idx, x_gcc6_2_32_3.shape[1])
y_gcc6_2_32_3 = y_gcc6_2_32_3.reshape(-1, right_idx, 1)

print(x_gcc6_2_32_3.shape, y_gcc6_2_32_3.shape)

(3533184, 256) (3533184,)
(73608, 48, 256) (73608, 48, 1)


## (7) 모델

In [8]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = Bidirectional(LSTM(48, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

Using TensorFlow backend.


## (8) 학습 - 10 KFold

In [9]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_2_32_3, y_gcc6_2_32_3):
    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_gcc6_2_32_3[train],
               y_gcc6_2_32_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_2_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_2_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
66247/66247 [==============================] - 129s 2ms/step - loss: 0.0082 - accuracy: 0.9983
Epoch 2/10
66247/66247 [==============================] - 141s 2ms/step - loss: 0.0037 - accuracy: 0.9989
Epoch 3/10
66247/66247 [=========================

accuracy_score 0.9991594212742834
recall_score 0.6728016359918201
precision_score 0.7060085836909872
f1_score 0.6890052356020944
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66247/66247 [==============================] - 144s 2ms/step - loss: 0.0018 - accuracy: 0.9994
Epoch 2/10
66247/66247 [==============================] - 143s 2ms/step - loss: 0.0017 - accuracy: 0.9995
Epoch 3/10
6624

accuracy_score 0.9993575374722637
recall_score 0.7222222222222222
precision_score 0.8351177730192719
f1_score 0.7745779543197616
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66247/66247 [==============================] - 151s 2ms/step - loss: 8.6242e-04 - accuracy: 0.9997
Epoch 2/10
66247/66247 [==============================] - 149s 2ms/step - loss: 7.3794e-04 - accuracy: 0.9998
Epoch 3

accuracy_score 0.9995698048272427
recall_score 0.8584905660377359
precision_score 0.8552631578947368
f1_score 0.856873822975518
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66247/66247 [==============================] - 150s 2ms/step - loss: 4.3084e-04 - accuracy: 0.9999
Epoch 2/10
66247/66247 [==============================] - 150s 2ms/step - loss: 3.5781e-04 - accuracy: 0.9999
Epoch 3/

accuracy_score 0.9997990535706199
recall_score 0.9260042283298098
precision_score 0.9240506329113924
f1_score 0.9250263991552271
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66247/66247 [==============================] - 152s 2ms/step - loss: 2.0670e-04 - accuracy: 0.9999
Epoch 2/10
66247/66247 [==============================] - 148s 2ms/step - loss: 1.5663e-04 - accuracy: 1.0000
Epoch 3

...

accuracy_score 0.999909432595209
recall_score 0.9530956848030019
precision_score 0.9864077669902913
f1_score 0.9694656488549619
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66247/66247 [==============================] - 93s 1ms/step - loss: 9.0845e-05 - accuracy: 1.0000
Epoch 2/10
66247/66247 [==============================] - 90s 1ms/step - loss: 4.5684e-05 - accuracy: 1.0000
Epoch 3/10

accuracy_score 0.9999547162976045
recall_score 0.974
precision_score 0.9938775510204082
f1_score 0.9838383838383838
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66247/66247 [==============================] - 92s 1ms/step - loss: 4.1069e-05 - accuracy: 1.0000A: - ETA: 18s - - ETA: 
Epoch 2/10
66247/66247 [==============================] - 91s 1ms/step - loss: 2.7869e-05 - accuracy: 1.0000

accuracy_score 0.999980188380202
recall_score 0.9912739965095986
precision_score 0.9964912280701754
f1_score 0.9938757655293089
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66248/66248 [==============================] - 95s 1ms/step - loss: 3.3515e-05 - accuracy: 1.0000
Epoch 2/10
66248/66248 [==============================] - 90s 1ms/step - loss: 1.9273e-05 - accuracy: 1.0000 0s - loss:

accuracy_score 0.9999943387681159
recall_score 0.9978858350951374
precision_score 0.9978858350951374
f1_score 0.9978858350951374
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 96)            117120    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
66248/66248 [==============================] - 102s 2ms/step - loss: 3.0115e-05 - accuracy: 1.0000
Epoch 2/10
66248/66248 [==============================] - 105s 2ms/step - loss: 2.5961e-05 - accuracy: 1.0000
Epoch 

accuracy_score 0.9999915081521739
recall_score 0.9981549815498155
precision_score 0.996316758747698
f1_score 0.9972350230414747

K-fold cross validation Accuracy: [0.9991594212742834, 0.9993575374722637, 0.9995698048272427, 0.9997990535706199, 0.9998811302812118, 0.999909432595209, 0.9999547162976045, 0.999980188380202, 0.9999943387681159, 0.9999915081521739]

K-fold cross validation Recall: [0.6728016359918201, 0.7222222222222222, 0.8584905660377359, 0.9260042283298098, 0.9381443298969072, 0.9530956848030019, 0.974, 0.9912739965095986, 0.9978858350951374, 0.9981549815498155]

K-fold cross validation Precision: [0.7060085836909872, 0.8351177730192719, 0.8552631578947368, 0.9240506329113924, 0.974304068522484, 0.9864077669902913, 0.9938775510204082, 0.9964912280701754, 0.9978858350951374, 0.996316758747698]

K-fold cross validation F1-Score: [0.6890052356020944, 0.7745779543197616, 0.856873822975518, 0.9250263991552271, 0.9558823529411765, 0.9694656488549619, 0.9838383838383838, 0.99387

## (9) 평가지표

In [10]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9997597131618928
10-Fold Cross_validation. Recall : 0.9032073480436049
10-Fold Cross_validation. Precision : 0.9265723355962583
10-Fold Cross_validation. F1-Score : 0.9143666421353045
